# Setup and data loading

In [1]:
import sys
sys.path.append("../")  # path contains python_file.py

In [2]:
import iris

import numpy as np
import scipy.stats

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
# Given an iris-cube this returns the dates stored in the last time-dimension
def get_dates(x):
    time_dimension = x.coords()[2]
    dates = time_dimension.units.num2date(time_dimension.points)
    return dates

get_dates = np.vectorize(get_dates)

# This reads in the testing-data from ISIMIP stored in testing_data/ISIMIP
def read_in_and_preprocess_isimip_testing_data_with_dates(variable, isimip_data_path = "testing_data/ISIMIP/"):
    
    # Load in data
    obs = iris.load_cube(isimip_data_path+variable+"_obs-hist_coarse_1979-2014.nc")
    cm_hist = iris.load_cube(isimip_data_path+variable+"_sim-hist_coarse_1979-2014.nc")
    cm_future = iris.load_cube(isimip_data_path+variable+"_sim-fut_coarse_2065-2100.nc")

    # Extract dates
    dates = {
        "time_obs": get_dates(obs),
        "time_cm_hist": get_dates(cm_hist),
        "time_cm_future": get_dates(cm_future)
    }
    
    # Convert to np.array (from masked-array)
    obs = np.array(obs.data)
    cm_hist = np.array(cm_hist.data)
    cm_future = np.array(cm_future.data)

    # Move time to first axis (our convention)
    obs = np.moveaxis(obs, -1, 0)
    cm_hist = np.moveaxis(cm_hist, -1, 0)
    cm_future = np.moveaxis(cm_future, -1, 0)
    
    return obs, cm_hist, cm_future, dates

In [5]:
obs, cm_hist, cm_future, dates = read_in_and_preprocess_isimip_testing_data_with_dates("tas")

/home/jakobwes/anaconda3/envs/ibias4/lib/python3.9/site-packages/pyproj/__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


In [6]:
from PACKAGE_NAME.debias import QuantileMapping

temp_debiaser = QuantileMapping.from_variable(variable = "tas")
temp_debiaser

QuantileMapping(distribution=<scipy.stats._continuous_distns.norm_gen object at 0x7f11529c0fa0>, detrending='additive', variable='Daily mean near-surface air temperature')

# Show debiasers running through

In [7]:
from scipy.stats import norm, laplace

from PACKAGE_NAME.debias import QuantileMapping, QuantileDeltaMapping, ScaledDistributionMapping, LinearScaling, ECDFM, DeltaChange, CDFt

distribution = norm

temp_debiaser = DeltaChange.from_variable(variable = "tas")
debiased_cm1 = temp_debiaser.apply(obs, cm_hist, cm_future)

temp_debiaser = QuantileMapping(distribution=norm, detrending="no_detrending")
debiased_cm2 = temp_debiaser.apply(obs, cm_hist, cm_future)

temp_debiaser = QuantileMapping.from_variable(variable="tas")
debiased_cm3 = temp_debiaser.apply(obs, cm_hist, cm_future)

temp_debiaser = QuantileMapping.from_variable(variable="tas", detrending="additive", distribution = laplace)
debiased_cm4 = temp_debiaser.apply(obs, cm_hist, cm_future)

temp_debiaser = ECDFM.from_variable(variable = "tas", distribution = norm)
debiased_cm5 = temp_debiaser.apply(obs, cm_hist, cm_future)

temp_debiaser = QuantileDeltaMapping.from_variable("tas")
debiased_cm6 = temp_debiaser.apply(obs, cm_hist, cm_future)

temp_debiaser = ScaledDistributionMapping.from_variable(variable = "tas")
debiased_cm7 = temp_debiaser.apply(obs, cm_hist, cm_future)

temp_debiaser = LinearScaling(delta_type = "additive")
debiased_cm8 = temp_debiaser.apply(obs, cm_hist, cm_future)

temp_debiaser = CDFt.from_variable("tas")
debiased_cm9 = temp_debiaser.apply(obs, cm_hist, cm_future, **dates)

----- Running debiasing -----


100%|███████████████████████████████████████████| 4/4 [00:00<00:00, 5251.08it/s]


----- Running debiasing -----


100%|████████████████████████████████████████████| 4/4 [00:00<00:00, 358.27it/s]


----- Running debiasing -----


...

100%|█████████████████████████████████████████████| 4/4 [00:00<00:00, 99.09it/s]


----- Running debiasing -----


100%|███████████████████████████████████████████| 4/4 [00:00<00:00, 8077.62it/s]


----- Running debiasing -----


100%|█████████████████████████████████████████████| 4/4 [00:01<00:00,  2.35it/s]


# Show different ways to initialize debiasers

In [8]:
from scipy.stats import norm, laplace, gamma

from PACKAGE_NAME.debias import QuantileMapping
from PACKAGE_NAME.debias import QuantileDeltaMapping
from PACKAGE_NAME.variables import tas, pr
from PACKAGE_NAME.utils import gen_PrecipitationGammaLeftCensoredModel, gen_PrecipitationHurdleModel

# Using a string and the required argument: maps to default settings
temp_debiaser1 = QuantileMapping.from_variable(variable = "t2m", detrending="no_detrending")

# Using a string and the required argument: maps to default settings
temp_debiaser2 = QuantileMapping.from_variable(variable = "tas", detrending="additive")

# Using the variable class: maps to default settings
temp_debiaser3 = QuantileMapping.from_variable(variable=tas, detrending="additive")

# Using the variable class but overwriting the standard distribution
temp_debiaser4 = QuantileMapping.from_variable(variable=tas, detrending="additive", distribution = laplace)

# Using special initialisation for precipitation
precip_debiaser1 = QuantileMapping.for_precipitation(detrending = "multiplicative", precipitation_model_type = "censored", precipitation_censoring_value = 0.1)

# Specifying all the arguments and an own model for the distribution (child-class of StatisticalModel needing only fit, cdf and ppf method) 1
left_censored_model = gen_PrecipitationGammaLeftCensoredModel(censoring_value = 0.1)
precip_debiaser2 = QuantileMapping(detrending = "multiplicative", distribution = left_censored_model)

# Using the for_precipitation method but an own model for the distribution (child-class of StatisticalModel needing only fit, cdf and ppf method) 2
hurdle_model = gen_PrecipitationHurdleModel(distribution = gamma)
precip_debiaser3 = QuantileMapping.for_precipitation(detrending = "multiplicative", distribution = hurdle_model)

# Using a variable, but overwriting the distribution
hurdle_model = gen_PrecipitationHurdleModel(distribution = gamma)
precip_debiaser4 = QuantileMapping.from_variable(variable = "pr", detrending = "multiplicative", distribution = hurdle_model)


TypeError: __init__() got an unexpected keyword argument 'precipitation_censoring_value'

In [ ]:
temp_debiaser1 == temp_debiaser2 # no because of different delta_type

In [ ]:
temp_debiaser2 == temp_debiaser3 # yes

In [ ]:
temp_debiaser3 == temp_debiaser4 # no because of different distribution

In [ ]:
precip_debiaser1 == precip_debiaser2 # yes because same models, just differently specified

In [ ]:
precip_debiaser2 == precip_debiaser3 # no because of different model: hurdle vs. censored

In [ ]:
precip_debiaser3 == precip_debiaser4 # yes because same model, just differently specified